# OpenAI

In [4]:
# ==============================
# Load environment variables
# ==============================
from dotenv import dotenv_values

# Load variables from app.env
env_vars = dotenv_values("app.env")
print("Loaded ENV variables:", env_vars)

# Extract required variables
openai_api_key = env_vars["OPENAI_API_KEY"]
openai_api_base = env_vars.get("OPENAI_API_BASE")
openai_api_name = env_vars["OPENAI_API_Name"]

Loaded ENV variables: OrderedDict({'OPENAI_API_KEY': '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', 'OPENAI_API_BASE': 'https://openrouter.ai/api/v1', 'OPENAI_API_Name': 'gpt-4o'})


In [5]:
# ==============================
# Create LangChain Chat Model
# ==============================
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model=openai_api_name,
    api_key=openai_api_key,
    base_url=openai_api_base,
    temperature=0.7,
    max_tokens=100
)

# Listing

In [6]:
# ==============================
# Send a prompt to the model
# ==============================
task = "Explain the theory of relativity in simple terms."

response = chat.invoke(task)

print("\nAI Response:")
print(response.content)


AI Response:
The theory of relativity, developed by Albert Einstein, consists of two main parts: special relativity and general relativity. Here's a simplified explanation of each:

1. **Special Relativity:** 
   - **Concept**: This part of the theory deals with the physics of objects moving at constant speeds, particularly those moving close to the speed of light.
   - **Key Ideas**:
     - **Speed of Light**: The speed of light is the same for all observers, no matter


In [7]:
!pip install -U langchain langchain-core langchain-openai

^C



  Attempting uninstall: langchain-core

    Found existing installation: langchain-core 1.2.1

    Uninstalling langchain-core-1.2.1:

      Successfully uninstalled langchain-core-1.2.1

   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
  Attempting uninstall: langchain-openai
   ---------------------------------------- 0/2 [langchain-core]
    Found existing installation: langchain-openai 1.1.3
   ---------------------------------------- 0/2 [langchain-core]
    Uninstalling langchain-openai-1.1.3:
   ---------------------------------------- 0/2 [langchain-core]
   -------------------- ------------------- 1/2 [langchain-openai]
      Successfully uninstalled langchain-openai-1.1.3
   -------------------- ------------------- 1/2 [langchain-openai]
   ---------------------------

In [8]:
# ==============================
# Output Parser Example 
# ==============================

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

# Create output parser
output_parser = CommaSeparatedListOutputParser()

# Get formatting instructions
format_instructions = output_parser.get_format_instructions()

print("\nFormat Instructions:")
print(format_instructions)

# Create a prompt template using the parser
prompt = PromptTemplate(
    template=(
        "List 5 key concepts of the {type}.\n"
        "{format_instructions}"
    ),
    input_variables=["type"],
    partial_variables={
        "format_instructions": format_instructions
    }
)

# Format the prompt (provide the required 'type' variable)
formatted_prompt = prompt.format(type="theory of relativity")

print("\nGenerated Prompt:")
print(formatted_prompt)

# Send the formatted prompt to the chat model
response = chat.invoke(formatted_prompt)
print("\nAI Response:")
print(response.content)

# Parse the response using the output parser
parsed_output = output_parser.parse(response.content)
print("\nParsed Output:")
print(parsed_output)


Format Instructions:
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`

Generated Prompt:
List 5 key concepts of the theory of relativity.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`

AI Response:
Time dilation, length contraction, mass-energy equivalence, equivalence principle, spacetime curvature

Parsed Output:
['Time dilation', 'length contraction', 'mass-energy equivalence', 'equivalence principle', 'spacetime curvature']


# Date Time

In [9]:
from langchain_core.prompts import PromptTemplate

In [16]:
prompt = PromptTemplate(
    template = "When was {person} born?",
    input_variables = ["person"]
)   
formatted_prompt = prompt.format(person="Albert Einstein")
print("\nFormatted Prompt:")
print(formatted_prompt)


Formatted Prompt:
When was Albert Einstein born?


In [17]:
# Send the formatted prompt to the chat model
response = chat.invoke(formatted_prompt)
print("\nAI Response:")
print(response.content)


AI Response:
Albert Einstein was born on March 14, 1879.


In [18]:
format_instructions = "Replay with a date in the format YYYY-MM-DD." 

prompt = PromptTemplate(
    template = "When was {person} born?\n{format_instructions}",
    input_variables = ["person"],
    partial_variables = {
        "format_instructions": format_instructions
    }
)   
formatted_prompt = prompt.format(person="Albert Einstein")
print("\nFormatted Prompt:")
print(formatted_prompt)


Formatted Prompt:
When was Albert Einstein born?
Replay with a date in the format YYYY-MM-DD.


In [20]:
# Send the formatted prompt to the chat model
response = chat.invoke(formatted_prompt)
print("\nAI Response:")
print(response.content)


AI Response:
Albert Einstein was born on 1879-03-14.


# PyDantic

useful to order data as oop 

your output can be class with some properties 

In [22]:
! pip install pydantic

In [23]:
{
    "plate_name": "Shawerma",
    "ingredients": 
    ["Chicken", "Garlic Sauce", "Pickles", "Pita Bread"]
}

{'plate_name': 'Shawerma',
 'ingredients': ['Chicken', 'Garlic Sauce', 'Pickles', 'Pita Bread']}

In [ ]:
from pydantic import BaseModel , Field
from langchain_core.output_parsers import PydanticOutputParser

# useful to order data as oop
# must inherit from BaseModel

class Plate(BaseModel):
    plate_name : str = Field(
        description="The name of the dish or plate."
    )   
    ingredients : list[str] = Field(
        description="The list of ingredients in the dish."
    )

# Create Pydantic output parser
output_parser = PydanticOutputParser(pydantic_object=Plate)

# Get formatting instructions
format_instructions = output_parser.get_format_instructions()
print("\nFormat Instructions:")
print(format_instructions)
# Create a prompt template using the parser
prompt = PromptTemplate(
    template=(
        "Describe a popular dish called {dish}.\n"
        "{format_instructions}"
    ),
    input_variables=["dish"],
    partial_variables={
        "format_instructions": format_instructions
    }
)

# Format the prompt (provide the required 'dish' variable)
formatted_prompt = prompt.format(dish="Shawerma")
print("\nGenerated Prompt:")
print(formatted_prompt)

# Send the formatted prompt to the chat model
response = chat.invoke(formatted_prompt)
print("\nAI Response:")
print(response.content)


Format Instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"plate_name": {"description": "The name of the dish or plate.", "title": "Plate Name", "type": "string"}, "ingredients": {"description": "The list of ingredients in the dish.", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}}, "required": ["plate_name", "ingredients"]}
```

Generated Prompt:
Describe a popular dish called Shawerma.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "

In [26]:
responsed_parsed = output_parser.parse(response.content)
print("\nParsed Output:")
print(responsed_parsed)


Parsed Output:
plate_name='Shawerma' ingredients=['Marinated meat (usually chicken, beef, or lamb)', 'Pita bread or flatbread', 'Tahini sauce', 'Garlic sauce', 'Pickles', 'Tomatoes', 'Onions', 'Lettuce', 'Spices (such as cumin, paprika, and turmeric)']
